# Master pipeline notebook

This notebook is mainly meant to run the experimentation. 

The experimentation is divided into seperate parts according to machine learning process

The process involves: 
1. Data gathering and initial transformation
2. Data analysis
3. feature engineering 
4. Modelling 
5. Evaluation

In [1]:
import numpy as np
import pandas as pd
from data import gatherData
import plotly.graph_objects as go
from pre_process_data import processData
from feature import engineerFeaturesForTraining, splitData
from transform import transformData
from utils import load_config_file
from train import train
import torch
from LSTM_model import LSTM
from inference import makeInference
from evaluation import evaluateModellingResults

In [2]:
config = load_config_file("config.yml")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

## 1. Data gathering and transformation

This stage is mainly to gather the data. Combine them if necessary. Perform initial data wrangling, so that it is ready for 
data analysis

In [3]:
gather_data_obj = gatherData(config=config)

In [4]:
(
    lap_times_df,
    pit_stops_df,
    qualifying_df,
    races_df,
    results_df,
    sprint_results_df,
    status_df,
) = gather_data_obj.load_data()

In [5]:
pre_process_obj = processData(config=config)

master_lap_times_df = pre_process_obj.create_initial_dataset(
    lap_times_df=lap_times_df, races_df=races_df, results_df=results_df
)
master_lap_times_df

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId
0,841,20,1,1,98109,2011,1,1,Australian Grand Prix,2011-03-27,1,1
1,841,20,2,1,93006,2011,1,1,Australian Grand Prix,2011-03-27,1,1
2,841,20,3,1,92713,2011,1,1,Australian Grand Prix,2011-03-27,1,1
3,841,20,4,1,92803,2011,1,1,Australian Grand Prix,2011-03-27,1,1
4,841,20,5,1,92342,2011,1,1,Australian Grand Prix,2011-03-27,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
551737,1110,817,40,17,114361,2023,12,13,Belgian Grand Prix,2023-07-30,3,1
551738,1110,817,41,17,113367,2023,12,13,Belgian Grand Prix,2023-07-30,3,1
551739,1110,817,42,16,115247,2023,12,13,Belgian Grand Prix,2023-07-30,3,1
551740,1110,817,43,16,112115,2023,12,13,Belgian Grand Prix,2023-07-30,3,1


In [6]:
# check for nans in the data
master_lap_times_df.isna().any()

raceId          False
driverId        False
lap             False
position        False
milliseconds    False
year            False
round           False
circuitId       False
name            False
date            False
number          False
statusId        False
dtype: bool

In [7]:
driver_id = 20
race_id = 841

lap_times_selected_driver_race = master_lap_times_df[
    (master_lap_times_df["raceId"] == race_id)
    & (master_lap_times_df["driverId"] == driver_id)
]

In [8]:
## Lap times and pit stops

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=lap_times_selected_driver_race["lap"],
        y=lap_times_selected_driver_race["milliseconds"],
        mode="lines+markers",
        name="Lap times in milliseconds",
    )
)

fig.update_layout(
    title="Laptimes by each lap", xaxis_title="Lap number", yaxis_title="Total lap time"
)

- Two outliers in the data 
- this is mainly the pit stops that happen in between the race

In [9]:
master_laptimes_pitstop_df = pre_process_obj.add_pitstop_data(
    master_laptime_data=master_lap_times_df, pit_stop_data=pit_stops_df
)
master_laptimes_pitstop_df

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
0,841,20,1,1,98109,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
1,841,20,2,1,93006,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
2,841,20,3,1,92713,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
3,841,20,4,1,92803,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
4,841,20,5,1,92342,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551737,1110,817,40,17,114361,2023,12,13,Belgian Grand Prix,2023-07-30,3,1,0.0,False
551738,1110,817,41,17,113367,2023,12,13,Belgian Grand Prix,2023-07-30,3,1,0.0,False
551739,1110,817,42,16,115247,2023,12,13,Belgian Grand Prix,2023-07-30,3,1,0.0,False
551740,1110,817,43,16,112115,2023,12,13,Belgian Grand Prix,2023-07-30,3,1,0.0,False


In [10]:
driver_id = 20
circuit_id = 1

lap_times_selected_driver_circuit = master_laptimes_pitstop_df[
    (master_laptimes_pitstop_df["circuitId"] == circuit_id)
    & (master_laptimes_pitstop_df["driverId"] == driver_id)
]

In [11]:
lap_times_selected_driver_circuit

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
0,841,20,1,1,98109,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
1,841,20,2,1,93006,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
2,841,20,3,1,92713,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
3,841,20,4,1,92803,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
4,841,20,5,1,92342,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517395,1076,20,18,15,85367,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517396,1076,20,19,15,85503,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517397,1076,20,20,15,85398,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517398,1076,20,21,13,85505,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False


In [12]:
list_race_id = lap_times_selected_driver_circuit["raceId"].unique().tolist()

In [13]:
## Lap times and pit stops

fig = go.Figure()

for race_id in list_race_id:

    lap_times_race = lap_times_selected_driver_circuit[
        lap_times_selected_driver_circuit["raceId"] == race_id
    ]

    fig.add_trace(
        go.Scatter(
            x=lap_times_race["lap"],
            y=lap_times_race["milliseconds"],
            mode="lines+markers",
            name=f"race id: {race_id}",
        )
    )

    fig.update_layout(
        title="Laptimes by each lap",
        xaxis_title="Lap number",
        yaxis_title="Total lap time",
    )
fig.show()

In [14]:
fig = go.Figure()

# Add a box trace
fig.add_trace(
    go.Box(
        y=lap_times_selected_driver_circuit["milliseconds"], name="Lap times for Aus GP"
    )
)

# Customize the layout
fig.update_layout(
    title="Box Plot with plotly.graph_objects",
    yaxis_title="Values",
)

# Show the figure
fig.show()

In [15]:
lap_times_selected_driver_circuit[lap_times_selected_driver_circuit["raceId"] == 948]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
111775,948,20,1,1,96916,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111776,948,20,2,1,91664,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111777,948,20,3,1,92167,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111778,948,20,4,1,92014,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111779,948,20,5,1,92273,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111780,948,20,6,1,92204,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111781,948,20,7,1,92080,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111782,948,20,8,1,92289,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111783,948,20,9,1,92578,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False
111784,948,20,10,1,92571,2016,1,1,Australian Grand Prix,2016-03-20,5,1,0.0,False


In [16]:
lap_times_selected_driver_circuit[lap_times_selected_driver_circuit["raceId"] == 1]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
343523,1,20,1,2,99647,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343524,1,20,2,2,91173,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343525,1,20,3,2,89752,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343526,1,20,4,2,88999,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343527,1,20,5,2,88849,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343528,1,20,6,2,88424,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343529,1,20,7,2,88235,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343530,1,20,8,2,88140,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343531,1,20,9,2,88331,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False
343532,1,20,10,2,88970,2009,1,1,Australian Grand Prix,2009-03-29,15,4,0.0,False


In [17]:
master_laptimes_pitstop_df[
    (master_laptimes_pitstop_df["raceId"] == 948)
    & (master_laptimes_pitstop_df["lap"] == 18)
]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
111792,948,20,18,1,201551,2016,1,1,Australian Grand Prix,2016-03-20,5,1,1089312.0,True
111849,948,8,18,3,188276,2016,1,1,Australian Grand Prix,2016-03-20,7,5,1089365.0,True
111870,948,3,18,2,199132,2016,1,1,Australian Grand Prix,2016-03-20,6,1,1089303.0,True
111927,948,830,18,5,189398,2016,1,1,Australian Grand Prix,2016-03-20,33,1,1089372.0,True
111984,948,13,18,8,1243269,2016,1,1,Australian Grand Prix,2016-03-20,19,1,1089805.0,True
112041,948,1,18,7,1243047,2016,1,1,Australian Grand Prix,2016-03-20,44,1,1090087.0,True
112098,948,832,18,6,1242495,2016,1,1,Australian Grand Prix,2016-03-20,55,1,1089971.0,True
112155,948,807,18,10,1229799,2016,1,1,Australian Grand Prix,2016-03-20,27,1,1089155.0,True
112212,948,817,18,4,190570,2016,1,1,Australian Grand Prix,2016-03-20,3,1,1089822.0,True
112285,948,815,18,13,1221249,2016,1,1,Australian Grand Prix,2016-03-20,11,1,1087311.0,True


In [18]:
master_laptimes_pitstop_df[
    (master_laptimes_pitstop_df["raceId"] == 948)
    & (master_laptimes_pitstop_df["driverId"] == 825)
]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
112735,948,825,1,21,180423,2016,1,1,Australian Grand Prix,2016-03-20,20,1,27831.0,True
112736,948,825,2,21,106207,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112737,948,825,3,21,97229,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112738,948,825,4,21,94720,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112739,948,825,5,21,95346,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112740,948,825,6,21,103453,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112741,948,825,7,21,94917,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112742,948,825,8,21,95742,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112743,948,825,9,21,95633,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False
112744,948,825,10,21,97304,2016,1,1,Australian Grand Prix,2016-03-20,20,1,0.0,False


In [19]:
lap_times_selected_driver_circuit[lap_times_selected_driver_circuit["raceId"] == 900]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
72699,900,20,1,15,120977,2014,1,1,Australian Grand Prix,2014-03-16,1,5,0.0,False
72700,900,20,2,16,109947,2014,1,1,Australian Grand Prix,2014-03-16,1,5,0.0,False
72701,900,20,3,16,111460,2014,1,1,Australian Grand Prix,2014-03-16,1,5,0.0,False


In [20]:
lap_times_selected_driver_circuit[
    (lap_times_selected_driver_circuit["raceId"] == 948)
    & (lap_times_selected_driver_circuit["lap"] == 18)
]

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
111792,948,20,18,1,201551,2016,1,1,Australian Grand Prix,2016-03-20,5,1,1089312.0,True


In [21]:
## work on the outlier
lap17_time = lap_times_selected_driver_circuit[
    (lap_times_selected_driver_circuit["raceId"] == 948)
    & (lap_times_selected_driver_circuit["lap"] == 17)
]["milliseconds"].values[0]

lap20_time = lap_times_selected_driver_circuit[
    (lap_times_selected_driver_circuit["raceId"] == 948)
    & (lap_times_selected_driver_circuit["lap"] == 20)
]["milliseconds"].values[0]

lap_times_selected_driver_circuit.loc[
    (lap_times_selected_driver_circuit["raceId"] == 948)
    & (lap_times_selected_driver_circuit["lap"] == 18),
    "milliseconds",
] = (lap17_time + lap20_time) / 2

lap_times_selected_driver_circuit.loc[
    (lap_times_selected_driver_circuit["raceId"] == 948)
    & (lap_times_selected_driver_circuit["lap"] == 19),
    "milliseconds",
] = (lap17_time + lap20_time) / 2

In [22]:
fig = go.Figure()

for race_id in list_race_id:

    lap_times_race = lap_times_selected_driver_circuit[
        lap_times_selected_driver_circuit["raceId"] == race_id
    ]

    fig.add_trace(
        go.Scatter(
            x=lap_times_race["lap"],
            y=lap_times_race["milliseconds"],
            mode="lines+markers",
            name=f"race id: {race_id}",
        )
    )

    fig.update_layout(
        title="Laptimes by each lap",
        xaxis_title="Lap number",
        yaxis_title="Total lap time",
    )
fig.show()

In [23]:
fig = go.Figure()

# Add a box trace
fig.add_trace(
    go.Box(
        y=lap_times_selected_driver_circuit["milliseconds"], name="Lap times for Aus GP"
    )
)

# Customize the layout
fig.update_layout(
    title="Box Plot with plotly.graph_objects",
    yaxis_title="Values",
)

# Show the figure
fig.show()

In [24]:
lap_times_selected_driver_circuit

,raceId,driverId,lap,position,milliseconds,year,round,circuitId,name,date,number,statusId,pitStopMilliseconds,isPitStop
0,841,20,1,1,98109.0,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
1,841,20,2,1,93006.0,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
2,841,20,3,1,92713.0,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
3,841,20,4,1,92803.0,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
4,841,20,5,1,92342.0,2011,1,1,Australian Grand Prix,2011-03-27,1,1,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517395,1076,20,18,15,85367.0,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517396,1076,20,19,15,85503.0,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517397,1076,20,20,15,85398.0,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False
517398,1076,20,21,13,85505.0,2022,3,1,Australian Grand Prix,2022-04-10,5,3,0.0,False


# Feature engineering

In [25]:
engineer_data_obj = engineerFeaturesForTraining(config=config)


engineered_lap_times_df, encoder, encoder_columns = engineer_data_obj.engineer_data(
    lap_times_data=lap_times_selected_driver_circuit
)

In [26]:
engineered_lap_times_df

,raceId,lap,position,milliseconds,year,milliseconds_1_prior,lap_number_1_prior,position_1_prior_lap,month,day,isPitStop_False,isPitStop_True,statusId_1,statusId_3,statusId_4,statusId_36
1,1,2,2,91173.0,2009,99647.0,1.0,2.0,3,29,1.0,0.0,0.0,0.0,1.0,0.0
2,1,3,2,89752.0,2009,91173.0,2.0,2.0,3,29,1.0,0.0,0.0,0.0,1.0,0.0
3,1,4,2,88999.0,2009,89752.0,3.0,2.0,3,29,1.0,0.0,0.0,0.0,1.0,0.0
4,1,5,2,88849.0,2009,88999.0,4.0,2.0,3,29,1.0,0.0,0.0,0.0,1.0,0.0
5,1,6,2,88424.0,2009,88849.0,5.0,2.0,3,29,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,1076,18,15,85367.0,2022,85189.0,17.0,16.0,4,10,1.0,0.0,0.0,1.0,0.0,0.0
561,1076,19,15,85503.0,2022,85367.0,18.0,15.0,4,10,1.0,0.0,0.0,1.0,0.0,0.0
562,1076,20,15,85398.0,2022,85503.0,19.0,15.0,4,10,1.0,0.0,0.0,1.0,0.0,0.0
563,1076,21,13,85505.0,2022,85398.0,20.0,15.0,4,10,1.0,0.0,0.0,1.0,0.0,0.0


In [27]:
fig = go.Figure()

for race_id in list_race_id:

    lap_times_race = engineered_lap_times_df[
        engineered_lap_times_df["raceId"] == race_id
    ]

    fig.add_trace(
        go.Scatter(
            x=lap_times_race["lap"],
            y=lap_times_race["milliseconds"],
            mode="lines+markers",
            name=f"race id: {race_id}",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=lap_times_race["lap"],
            y=lap_times_race["milliseconds_1_prior"],
            mode="lines+markers",
            name=f"race id: {race_id} 1 prior",
        )
    )

    fig.update_layout(
        title="Laptimes by each lap and prior lap",
        xaxis_title="Lap number",
        yaxis_title="Total lap time",
    )
fig.show()

# Spliting into train test and validation

In [28]:
split_obj = splitData()

train_data, test_data = split_obj.train_test_split(
    data=engineered_lap_times_df, race_id=338
)

training_set, validation_set = split_obj.train_validation_split(train_data=train_data)

# Transform data into modelling data
- scaling 
- sequencing

In [29]:
## training_set trainformation

In [30]:
data_transform_obj = transformData(
    n_steps_input=config.get("N_STEP_INPUT"),
    n_steps_output=config.get("N_STEP_OUTPUT"),
    config=config,
)

scaled_input_df_train, scaled_output_df_train = (
    data_transform_obj.create_scaled_input_output_data(data=training_set, train=True)
)


scaled_input_df_val, scaled_output_df_val = (
    data_transform_obj.create_scaled_input_output_data(data=validation_set, train=False)
)

INFO:root:scaler_dict.pkl saved to ../artifacts/


In [31]:
scaled_input_df_val

,year_scaled,day_scaled,lap_scaled,position_scaled,milliseconds_1_prior_scaled,lap_number_1_prior_scaled,position_1_prior_lap_scaled,month_sin,month_cos,isPitStop_False,statusId_3,statusId_36,statusId_1,isPitStop_True,raceId,statusId_4
0,-1.660550,1.495531,1.000000,0.055556,0.273641,1.000000,0.055556,1.000000,6.123234e-17,1.0,0.0,0.0,0.0,0.0,1,1.0
1,-1.660550,1.495531,1.022727,0.055556,0.112934,1.022727,0.055556,1.000000,6.123234e-17,1.0,0.0,0.0,0.0,0.0,1,1.0
2,-1.660550,1.495531,1.045455,0.055556,0.055577,1.045455,0.055556,1.000000,6.123234e-17,1.0,0.0,0.0,0.0,0.0,1,1.0
3,-1.660550,1.495531,1.068182,0.055556,0.051616,1.068182,0.055556,1.000000,6.123234e-17,1.0,0.0,0.0,0.0,0.0,1,1.0
4,-1.660550,1.495531,1.090909,0.055556,0.052558,1.090909,0.055556,1.000000,6.123234e-17,1.0,0.0,0.0,0.0,0.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2.102348,-2.096095,0.363636,0.777778,-0.004709,0.363636,0.833333,0.866025,-5.000000e-01,1.0,1.0,0.0,0.0,0.0,1076,0.0
108,2.102348,-2.096095,0.386364,0.777778,-0.002412,0.386364,0.777778,0.866025,-5.000000e-01,1.0,1.0,0.0,0.0,0.0,1076,0.0
109,2.102348,-2.096095,0.409091,0.777778,-0.000658,0.409091,0.777778,0.866025,-5.000000e-01,1.0,1.0,0.0,0.0,0.0,1076,0.0
110,2.102348,-2.096095,0.431818,0.666667,-0.002013,0.431818,0.777778,0.866025,-5.000000e-01,1.0,1.0,0.0,0.0,0.0,1076,0.0


In [32]:
x_sequential_train, y_sequential_train = data_transform_obj.create_sequence(
    scaled_input_df=scaled_input_df_train, scaled_output_df=scaled_output_df_train
)

x_sequential_val, y_sequential_val = data_transform_obj.create_sequence(
    scaled_input_df=scaled_input_df_val, scaled_output_df=scaled_output_df_val
)

In [33]:
x_train = torch.Tensor(x_sequential_train)
y_train = torch.Tensor(y_sequential_train)

x_train = torch.reshape(
    x_train,
    (
        x_train.shape[0],
        2,
        x_train.shape[2],
    ),
)

x_validation = torch.Tensor(x_sequential_val)
y_validation = torch.Tensor(y_sequential_val)

x_validation = torch.reshape(
    x_validation,
    (
        x_validation.shape[0],
        2,
        x_validation.shape[2],
    ),
)

In [34]:
train_obj = train(config=config)

train_data_loader, validation_data_loader = train_obj.create_data_loader(
    batch_size=8,
    x_train=x_train,
    y_train=y_train,
    x_validation=x_validation,
    y_validation=y_validation,
)

In [35]:
x_train.shape[2]

15

In [36]:
lstm_model = LSTM(
    num_classes=config.get("N_STEP_OUTPUT"),
    input_size=x_train.shape[2],
    hidden_size_layer=8,
    num_layers=config.get("NUM_LAYERS"),
    dropout_rate=0.1,
)
lstm_model.to(device)

loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

validation_loss = train_obj.train_validation_loop(
    n_epochs=10,
    lstm_model=lstm_model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_data_loader,
    validation_loader=validation_data_loader,
    device=device,
)

Epoch num: 0
Batch 1, train loss: 0.003481788095086813
Batch 2, train loss: 0.0023195818066596985
Batch 3, train loss: 0.0027585846837610006
Batch 4, train loss: 0.0030385381542146206
Batch 5, train loss: 0.0030039858538657427
Batch 6, train loss: 0.0025531125720590353
Batch 7, train loss: 0.0029199616983532906
Batch 8, train loss: 0.002176359063014388
Batch 9, train loss: 0.0025408812798559666
Batch 10, train loss: 0.002022148109972477
Batch 11, train loss: 0.002903018379583955
Batch 12, train loss: 0.002345136133953929
Batch 13, train loss: 0.002488859696313739
Batch 14, train loss: 0.0015431619249284267
Batch 15, train loss: 0.001419375534169376
Batch 16, train loss: 0.002274208003655076
Batch 17, train loss: 0.0018381415866315365
Batch 18, train loss: 0.0011922772973775864
Batch 19, train loss: 0.0015109314117580652
Batch 20, train loss: 0.0018862431170418859
Batch 21, train loss: 0.001817969954572618
Batch 22, train loss: 0.0018343963893130422
Batch 23, train loss: 0.0020695233251

INFO:root:model.pkl saved to ../artifacts/


Validation loss: 0.019556058570742607
Epoch num: 9
Batch 1, train loss: 6.381179264280945e-05
Batch 2, train loss: 7.698973058722913e-05
Batch 3, train loss: 0.00010303844464942813
Batch 4, train loss: 0.0003459042636677623
Batch 5, train loss: 0.00017213408136740327
Batch 6, train loss: 0.00011435957276262343
Batch 7, train loss: 0.000855686201248318
Batch 8, train loss: 0.00032394513254985213
Batch 9, train loss: 0.0003720449167303741
Batch 10, train loss: 0.00012250131112523377
Batch 11, train loss: 0.0007979045622050762
Batch 12, train loss: 0.0005879512173123658
Batch 13, train loss: 7.441580237355083e-05
Batch 14, train loss: 0.00015088316285982728
Batch 15, train loss: 0.00013092553126625717
Batch 16, train loss: 0.0002626929781399667
Batch 17, train loss: 0.00010806929640239105
Batch 18, train loss: 0.00025784660829231143
Batch 19, train loss: 0.0007134546176530421
Batch 20, train loss: 7.424324576277286e-05
Batch 21, train loss: 5.77874670852907e-05
Batch 22, train loss: 8.422

In [37]:
infer_obj = makeInference(config=config)

prediction_tensor, actual_sequential, lap_sequential = infer_obj.perform_inference(
    test_data=test_data, device=device
)

tensor([[0.6335, 0.0000, 0.5170, 0.0000, 0.0426],
        [0.6592, 0.0000, 0.4992, 0.0017, 0.0759],
        [0.5794, 0.0000, 0.4792, 0.0304, 0.0967],
        [0.1101, 0.0000, 0.3951, 0.1928, 0.2390],
        [0.0000, 0.0000, 0.3672, 0.2012, 0.2237],
        [0.0000, 0.0000, 0.3552, 0.2121, 0.2304],
        [0.0000, 0.0000, 0.3187, 0.2378, 0.2415],
        [0.1135, 0.0000, 0.2791, 0.1846, 0.1884],
        [0.0000, 0.0000, 0.2815, 0.2471, 0.2596],
        [0.0000, 0.0000, 0.2925, 0.2799, 0.2824],
        [0.0000, 0.0000, 0.2822, 0.2870, 0.2811],
        [0.0000, 0.0000, 0.2702, 0.2978, 0.2884],
        [0.0000, 0.0000, 0.2599, 0.3058, 0.2933],
        [0.0000, 0.0000, 0.2521, 0.3104, 0.2956],
        [0.0000, 0.0000, 0.2396, 0.3240, 0.3063],
        [0.0000, 0.0000, 0.2276, 0.3342, 0.3127],
        [0.0000, 0.0000, 0.2170, 0.3430, 0.3184],
        [0.0000, 0.0000, 0.2065, 0.3519, 0.3243],
        [0.0000, 0.0000, 0.1962, 0.3607, 0.3302]])


In [38]:
prediction_tensor.tolist()

[[134528.83568680286, 85189.0, 125453.07582777739, 85189.0, 88507.9287757352],
 [136523.17064720392,
  85189.0,
  124067.6997371912,
  85318.03750997037,
  91097.45458186418],
 [130313.07228171825,
  85189.0,
  122505.52337101102,
  87555.05710465461,
  92722.98428342491],
 [93763.47914589942,
  85189.0,
  115955.17006096244,
  100204.3509016335,
  103804.71241012216],
 [85189.0,
  85189.0,
  113787.39095491171,
  100855.08500164747,
  102611.86352217197],
 [85189.0,
  85189.0,
  112849.03234091401,
  101705.01214468479,
  103131.82228213549],
 [85189.0, 85189.0, 110008.6439448595, 103704.98707449436, 103998.60898804665],
 [94025.16096109152,
  85189.0,
  106928.11586368084,
  99568.18556210399,
  99863.25576409698],
 [85189.0,
  85189.0,
  107115.82240110636,
  104430.27785652876,
  105407.34569266438],
 [85189.0,
  85189.0,
  107969.29831501842,
  106987.40749269724,
  107181.94930168986],
 [85189.0,
  85189.0,
  107163.10994708538,
  107536.63374823332,
  107083.04559123516],
 [8518

In [39]:
actual_sequential

[array([156229., 150999., 104978., 103250., 102775.]),
 array([150999., 104978., 103250., 102775., 101476.]),
 array([104978., 103250., 102775., 101476., 121768.]),
 array([103250., 102775., 101476., 121768., 103572.]),
 array([102775., 101476., 121768., 103572.,  93057.]),
 array([101476., 121768., 103572.,  93057.,  92439.]),
 array([121768., 103572.,  93057.,  92439.,  91856.]),
 array([103572.,  93057.,  92439.,  91856.,  92287.]),
 array([93057., 92439., 91856., 92287., 93925.]),
 array([92439., 91856., 92287., 93925., 92447.]),
 array([91856., 92287., 93925., 92447., 91866.]),
 array([92287., 93925., 92447., 91866., 91812.]),
 array([93925., 92447., 91866., 91812., 91744.]),
 array([92447., 91866., 91812., 91744., 91626.]),
 array([91866., 91812., 91744., 91626., 91558.]),
 array([91812., 91744., 91626., 91558., 91556.]),
 array([91744., 91626., 91558., 91556., 92064.]),
 array([91626., 91558., 91556., 92064., 92939.]),
 array([91558., 91556., 92064., 92939., 92391.])]

In [40]:
lap_sequential

[array([3, 4, 5, 6, 7], dtype=int64),
 array([4, 5, 6, 7, 8], dtype=int64),
 array([5, 6, 7, 8, 9], dtype=int64),
 array([ 6,  7,  8,  9, 10], dtype=int64),
 array([ 7,  8,  9, 10, 11], dtype=int64),
 array([ 8,  9, 10, 11, 12], dtype=int64),
 array([ 9, 10, 11, 12, 13], dtype=int64),
 array([10, 11, 12, 13, 14], dtype=int64),
 array([11, 12, 13, 14, 15], dtype=int64),
 array([12, 13, 14, 15, 16], dtype=int64),
 array([13, 14, 15, 16, 17], dtype=int64),
 array([14, 15, 16, 17, 18], dtype=int64),
 array([15, 16, 17, 18, 19], dtype=int64),
 array([16, 17, 18, 19, 20], dtype=int64),
 array([17, 18, 19, 20, 21], dtype=int64),
 array([18, 19, 20, 21, 22], dtype=int64),
 array([19, 20, 21, 22, 23], dtype=int64),
 array([20, 21, 22, 23, 24], dtype=int64),
 array([21, 22, 23, 24, 25], dtype=int64)]

# Evaluation

In [41]:
evaluate = evaluateModellingResults()

rmse_mae_mape_dict = evaluate.get_model_evaluation_metrics(
    ls_prediction=prediction_tensor.tolist(),
    ls_actual=actual_sequential,
    lap_sequential=lap_sequential,
)

[134528.83568680286, 85189.0, 125453.07582777739, 85189.0, 88507.9287757352]
[136523.17064720392, 85189.0, 124067.6997371912, 85318.03750997037, 91097.45458186418]
[130313.07228171825, 85189.0, 122505.52337101102, 87555.05710465461, 92722.98428342491]
[93763.47914589942, 85189.0, 115955.17006096244, 100204.3509016335, 103804.71241012216]
[85189.0, 85189.0, 113787.39095491171, 100855.08500164747, 102611.86352217197]
[85189.0, 85189.0, 112849.03234091401, 101705.01214468479, 103131.82228213549]
[85189.0, 85189.0, 110008.6439448595, 103704.98707449436, 103998.60898804665]
[94025.16096109152, 85189.0, 106928.11586368084, 99568.18556210399, 99863.25576409698]
[85189.0, 85189.0, 107115.82240110636, 104430.27785652876, 105407.34569266438]
[85189.0, 85189.0, 107969.29831501842, 106987.40749269724, 107181.94930168986]
[85189.0, 85189.0, 107163.10994708538, 107536.63374823332, 107083.04559123516]
[85189.0, 85189.0, 106228.87802675366, 108383.19779840112, 107651.65895149112]
[85189.0, 85189.0, 10

In [42]:
rmse_mae_mape_df = pd.DataFrame(rmse_mae_mape_dict)


In [43]:
fig = go.Figure()

fig.add_trace(
        go.Scatter(
            x=rmse_mae_mape_df["current_lap"],
            y=rmse_mae_mape_df["RMSE"],
            mode="lines+markers",
            name=f"RMSE",
        )
    )

fig.add_trace(
        go.Scatter(
            x=rmse_mae_mape_df["current_lap"],
            y=rmse_mae_mape_df["MAE"],
            mode="lines+markers",
            name=f"MAE",
        )
    )


fig.update_layout(
        title="error rate in prediction at each lap",
        xaxis_title="Lap number",
        yaxis_title="Error",
    )